In [1]:
import cv2
import os
import sys
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import glob
import imutils
from imutils.video import VideoStream

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=sys.maxsize)


physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass


In [2]:
def image_capture(name):
    n=0
    print('Press C to Capture...')
    detector=cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    vid=VideoStream(src=0).start()
    while True:
        frame=vid.read()
        temp=frame.copy()
        frame=imutils.resize(frame, width=400)
        rects=detector.detectMultiScale(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        for (x, y, w, h) in rects:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)    
        cv2.imshow("Facial Recognition", frame)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("c"):
            cv2.imwrite("./images/"+name+".jpg",frame[y:y+h,x:x+w])
            n+=1
        elif key==ord("q") or n==1:
            break        
    print("{} Image has been stored in the dataset!".format(n))
    cv2.destroyAllWindows()
    vid.stop()

In [3]:
num = int(input('How many people do you want to enroll : '))
for i in range(num):
    name = input('Enter the name of the person to Enroll : ')
    image_capture(name)

How many people do you want to enroll : 0


In [4]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

In [5]:
weights = utils.weights
weights_dict = utils.load_weights()

for name in weights:
  if model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

In [6]:
def image_to_embedding(image, model):
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [7]:
def recognize_face(face_image, input_embeddings, model):
    embedding = image_to_embedding(face_image, model)
    minimum_distance = 200
    name = None
    for (input_name, input_embedding) in input_embeddings.items():   
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    if minimum_distance < 0.68:
        return str(name)
    else:
        return None

In [8]:
def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings


In [9]:
def recognize_faces_in_cam(input_embeddings):
    cv2.namedWindow("Deep Face Recognition")
#     vc = cv2.VideoCapture(0)
    vc = cv2.VideoCapture('classofmorons.mkv')
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    frame_width = int(vc.get(3)) 
    frame_height = int(vc.get(4)) 
    size = (frame_width, frame_height) 
    result = cv2.VideoWriter('result.avi',  cv2.VideoWriter_fourcc(*'MJPG'), 10, size) 
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h    
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, model)
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
            else:
                identity = 'Unknown'
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        result.write(img)
#         key = cv2.waitKey(100)
#         cv2.imshow("Face Recognizer", img)
         
#         if key == 27: # exit on ESC
#             break
    vc.release()
    cv2.destroyAllWindows()

In [10]:
input_embeddings = create_input_image_embeddings()
recognize_faces_in_cam(input_embeddings)

Euclidean distance from Unnikrishnan is 0.5297856
Euclidean distance from Shobit is 0.70457345
Euclidean distance from Arun is 0.9903283
Euclidean distance from Atharva is 0.890127
Euclidean distance from Unnikrishnan is 0.69170123
Euclidean distance from Shobit is 0.7074603
Euclidean distance from Arun is 0.8487657
Euclidean distance from Atharva is 0.66005164
Euclidean distance from Unnikrishnan is 0.6749202
Euclidean distance from Shobit is 0.7508273
Euclidean distance from Arun is 0.855563
Euclidean distance from Atharva is 0.76282805
Euclidean distance from Unnikrishnan is 0.5768848
Euclidean distance from Shobit is 0.5585798
Euclidean distance from Arun is 0.95882934
Euclidean distance from Atharva is 0.8832208
Euclidean distance from Unnikrishnan is 0.516023
Euclidean distance from Shobit is 0.7033455
Euclidean distance from Arun is 0.9704685
Euclidean distance from Atharva is 0.8659371
Euclidean distance from Unnikrishnan is 0.69170123
Euclidean distance from Shobit is 0.707460

Euclidean distance from Unnikrishnan is 0.75238
Euclidean distance from Shobit is 0.7010762
Euclidean distance from Arun is 0.8911223
Euclidean distance from Atharva is 0.69178355
Euclidean distance from Unnikrishnan is 0.63932335
Euclidean distance from Shobit is 0.85650915
Euclidean distance from Arun is 0.69791144
Euclidean distance from Atharva is 0.6944121
Euclidean distance from Unnikrishnan is 0.76951295
Euclidean distance from Shobit is 0.8310726
Euclidean distance from Arun is 0.8634994
Euclidean distance from Atharva is 0.7757589
Euclidean distance from Unnikrishnan is 0.4870488
Euclidean distance from Shobit is 0.46698728
Euclidean distance from Arun is 0.77548
Euclidean distance from Atharva is 0.7167204
Euclidean distance from Unnikrishnan is 0.59114915
Euclidean distance from Shobit is 0.72487384
Euclidean distance from Arun is 0.9153204
Euclidean distance from Atharva is 0.81069636
Euclidean distance from Unnikrishnan is 0.44589397
Euclidean distance from Shobit is 0.746

Euclidean distance from Unnikrishnan is 0.5225385
Euclidean distance from Shobit is 0.7446974
Euclidean distance from Arun is 0.98000556
Euclidean distance from Atharva is 0.8591067
Euclidean distance from Unnikrishnan is 0.43970525
Euclidean distance from Shobit is 0.7354047
Euclidean distance from Arun is 0.86068785
Euclidean distance from Atharva is 0.65839875
Euclidean distance from Unnikrishnan is 0.7487741
Euclidean distance from Shobit is 0.8596373
Euclidean distance from Arun is 0.97361
Euclidean distance from Atharva is 0.9337227
Euclidean distance from Unnikrishnan is 0.74397016
Euclidean distance from Shobit is 0.93309635
Euclidean distance from Arun is 0.9311863
Euclidean distance from Atharva is 0.7990743
Euclidean distance from Unnikrishnan is 0.62208927
Euclidean distance from Shobit is 0.6095824
Euclidean distance from Arun is 1.0131135
Euclidean distance from Atharva is 0.96463394
Euclidean distance from Unnikrishnan is 0.5608592
Euclidean distance from Shobit is 0.737

Euclidean distance from Unnikrishnan is 0.5627567
Euclidean distance from Shobit is 0.72800165
Euclidean distance from Arun is 0.96468043
Euclidean distance from Atharva is 0.8229235
Euclidean distance from Unnikrishnan is 0.840083
Euclidean distance from Shobit is 0.9067635
Euclidean distance from Arun is 1.089581
Euclidean distance from Atharva is 0.9008909
Euclidean distance from Unnikrishnan is 0.728239
Euclidean distance from Shobit is 0.53319514
Euclidean distance from Arun is 0.8967842
Euclidean distance from Atharva is 0.9134585
Euclidean distance from Unnikrishnan is 0.9296305
Euclidean distance from Shobit is 0.69045025
Euclidean distance from Arun is 0.70878816
Euclidean distance from Atharva is 0.7755021
Euclidean distance from Unnikrishnan is 0.56378174
Euclidean distance from Shobit is 0.7277112
Euclidean distance from Arun is 0.96445143
Euclidean distance from Atharva is 0.82324594
Euclidean distance from Unnikrishnan is 0.7721298
Euclidean distance from Shobit is 0.8171

Euclidean distance from Unnikrishnan is 0.9725743
Euclidean distance from Shobit is 0.829926
Euclidean distance from Arun is 0.70510674
Euclidean distance from Atharva is 0.78799117
Euclidean distance from Unnikrishnan is 0.53862935
Euclidean distance from Shobit is 0.7398687
Euclidean distance from Arun is 0.92252344
Euclidean distance from Atharva is 0.7853042
Euclidean distance from Unnikrishnan is 0.81652933
Euclidean distance from Shobit is 0.8485354
Euclidean distance from Arun is 0.9782906
Euclidean distance from Atharva is 0.8312334
Euclidean distance from Unnikrishnan is 0.67013615
Euclidean distance from Shobit is 0.44060305
Euclidean distance from Arun is 0.8071214
Euclidean distance from Atharva is 0.83261234
Euclidean distance from Unnikrishnan is 0.9465791
Euclidean distance from Shobit is 0.8263003
Euclidean distance from Arun is 0.6633409
Euclidean distance from Atharva is 0.7499481
Euclidean distance from Unnikrishnan is 0.47561967
Euclidean distance from Shobit is 0.7

...

Euclidean distance from Unnikrishnan is 0.769262
Euclidean distance from Shobit is 0.8812675
Euclidean distance from Arun is 0.81000656
Euclidean distance from Atharva is 0.6382176
Euclidean distance from Unnikrishnan is 0.636399
Euclidean distance from Shobit is 0.435699
Euclidean distance from Arun is 0.7986801
Euclidean distance from Atharva is 0.7892505
Euclidean distance from Unnikrishnan is 0.4981027
Euclidean distance from Shobit is 0.7063323
Euclidean distance from Arun is 1.0162067
Euclidean distance from Atharva is 0.8370992
Euclidean distance from Unnikrishnan is 0.8122532
Euclidean distance from Shobit is 0.7788228
Euclidean distance from Arun is 0.3541003
Euclidean distance from Atharva is 0.61102676
Euclidean distance from Unnikrishnan is 0.7667681
Euclidean distance from Shobit is 0.88218355
Euclidean distance from Arun is 0.80422544
Euclidean distance from Atharva is 0.63485503
Euclidean distance from Unnikrishnan is 0.64583176
Euclidean distance from Shobit is 0.448568

Euclidean distance from Unnikrishnan is 0.41543722
Euclidean distance from Shobit is 0.6814645
Euclidean distance from Arun is 0.9149576
Euclidean distance from Atharva is 0.7646177
Euclidean distance from Unnikrishnan is 0.6694926
Euclidean distance from Shobit is 0.80847454
Euclidean distance from Arun is 0.9072789
Euclidean distance from Atharva is 0.71232337
Euclidean distance from Unnikrishnan is 0.87200874
Euclidean distance from Shobit is 0.64373404
Euclidean distance from Arun is 1.0784774
Euclidean distance from Atharva is 1.0213115
Euclidean distance from Unnikrishnan is 0.4588505
Euclidean distance from Shobit is 0.6626454
Euclidean distance from Arun is 0.94739825
Euclidean distance from Atharva is 0.80361444
Euclidean distance from Unnikrishnan is 0.76459026
Euclidean distance from Shobit is 0.634196
Euclidean distance from Arun is 0.98765683
Euclidean distance from Atharva is 0.9832681
Euclidean distance from Unnikrishnan is 0.45677406
Euclidean distance from Shobit is 0.

Euclidean distance from Unnikrishnan is 0.72857887
Euclidean distance from Shobit is 0.60401636
Euclidean distance from Arun is 0.8403235
Euclidean distance from Atharva is 0.888405
Euclidean distance from Unnikrishnan is 0.6154332
Euclidean distance from Shobit is 0.7483027
Euclidean distance from Arun is 1.0218984
Euclidean distance from Atharva is 0.909193
Euclidean distance from Unnikrishnan is 0.6983969
Euclidean distance from Shobit is 0.78166276
Euclidean distance from Arun is 0.96921074
Euclidean distance from Atharva is 0.7839904
Euclidean distance from Unnikrishnan is 0.664071
Euclidean distance from Shobit is 0.6343742
Euclidean distance from Arun is 0.8143838
Euclidean distance from Atharva is 0.8613666
Euclidean distance from Unnikrishnan is 0.61432767
Euclidean distance from Shobit is 0.7498182
Euclidean distance from Arun is 1.0214084
Euclidean distance from Atharva is 0.9082989
Euclidean distance from Unnikrishnan is 0.72665936
Euclidean distance from Shobit is 0.765363

Euclidean distance from Unnikrishnan is 0.57569146
Euclidean distance from Shobit is 0.7032865
Euclidean distance from Arun is 0.9559859
Euclidean distance from Atharva is 0.8158814
Euclidean distance from Unnikrishnan is 0.5841554
Euclidean distance from Shobit is 0.8347228
Euclidean distance from Arun is 1.1213617
Euclidean distance from Atharva is 0.99923134
Euclidean distance from Unnikrishnan is 0.79488885
Euclidean distance from Shobit is 0.6903611
Euclidean distance from Arun is 0.76503545
Euclidean distance from Atharva is 0.8337698
Euclidean distance from Unnikrishnan is 1.0193292
Euclidean distance from Shobit is 1.2165779
Euclidean distance from Arun is 0.9736548
Euclidean distance from Atharva is 1.0513265
Euclidean distance from Unnikrishnan is 0.75186175
Euclidean distance from Shobit is 0.6024075
Euclidean distance from Arun is 0.90677583
Euclidean distance from Atharva is 0.9527929
Euclidean distance from Unnikrishnan is 0.5772986
Euclidean distance from Shobit is 0.703

AttributeError: 'NoneType' object has no attribute 'shape'